<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 3
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10
## <center> Прогнозирование задержек вылетов

Ваша задача – побить как минимум 2 бенчмарка в [соревновании](https://www.kaggle.com/c/flight-delays-spring-2018) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен второй – с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит градиентный бустинг. Скорее всего Xgboost, но тут у нас еще немало категориальных признаков...

<img src='../../img/xgboost_meme.jpg' width=40% />

In [0]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [3]:
!git clone https://github.com/Yorko/mlcourse_open.git

Cloning into 'mlcourse_open'...
remote: Counting objects: 5775, done.
remote: Total 5775 (delta 0), reused 0 (delta 0), pack-reused 5775
Receiving objects: 100% (5775/5775), 390.89 MiB | 33.97 MiB/s, done.
Resolving deltas: 100% (3254/3254), done.


In [4]:
!ls

datalab  mlcourse_open


In [0]:
train = pd.read_csv('mlcourse_open/data/flight_delays_train.csv')
test = pd.read_csv('mlcourse_open/data/flight_delays_test.csv')

In [4]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [5]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [0]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

In [0]:
scaler = StandardScaler()
X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.transform(X_valid)

In [115]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.6795691465352607

In [0]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logit.fit(X_train_scaled, y_train)
logit_test_pred = logit.predict_proba(X_test_scaled)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

In [83]:
!ls

datalab  logit_2feat.csv  mlcourse_open


Второй бенчмарк, представленный в рейтинге соревнования, был получен так:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, вы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [0]:
y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

In [0]:
#train['route'] = train['Origin'] + '-' + train['Dest']

In [0]:
categ_features = ['Month', 'DayOfWeek', 'DayofMonth', 'UniqueCarrier']
metric_features = ['Distance', 'DepTime']
features = categ_features + metric_features

In [0]:

train = train[features]
test = test[features]

In [0]:
data_df = pd.concat([train, test], axis=0, ignore_index=True)

In [120]:
data_df.shape

(200000, 6)

In [0]:
from sklearn import preprocessing

In [0]:
for feat in categ_features:
  data_df[feat] = preprocessing.LabelEncoder().fit_transform(data_df[feat])

In [145]:
data_df.head()

,Month,DayOfWeek,DayofMonth,UniqueCarrier,Distance,DepTime
0,10,6,13,1,732,1934
1,6,2,12,19,834,1548
2,11,4,11,21,416,1422
3,2,5,17,16,872,1015
4,1,5,28,20,423,1828


In [0]:
data_df = pd.get_dummies(data_df, columns=categ_features)

In [0]:
X_train, X_test = data_df[:100000], data_df[100000:]

In [124]:
X_train.shape, X_test.shape

((100000, 6), (100000, 6))

In [0]:
ss = StandardScaler()
X_train[metric_features] = ss.fit_transform(X_train[metric_features])
X_test[metric_features] = ss.transform(X_test[metric_features])

In [0]:
X_train_, X_valid_, y_train_, y_valid_ = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

In [0]:
model_xgb = xgboost.XGBClassifier(n_estimators=1000, max_depth=4, reg_alpha=1)

In [157]:
model_xgb.fit(X_train_, y_train_)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [158]:
xgb_valid_pred_ = model_xgb.predict_proba(X_valid_)[:, 1]

roc_auc_score(y_valid_, xgb_valid_pred_)

0.7288235828555915

In [0]:
model_test_pred = model_xgb.predict_proba(X_test)[:, 1]

pd.Series(model_test_pred, name='dep_delayed_15min').to_csv('model_2feat.csv', index_label='id', header=True)

In [0]:
!mv model_2feat.csv drive/model_3feat.csv

In [0]:
data_df_logit = pd.get_dummies(data_df, columns=categ_features)

In [0]:
X_train_logit, X_test_logit = data_df_logit[:100000], data_df_logit[100000:]

In [0]:
ss_logit = StandardScaler()
X_train_logit[metric_features] = ss_logit.fit_transform(X_train_logit[metric_features])
X_test_logit[metric_features] = ss_logit.transform(X_test_logit[metric_features])

In [0]:
X_train_lg, X_valid_lg, y_train_lg, y_valid_lg = train_test_split(X_train_logit, y_train, test_size=0.3, random_state=17)

In [0]:
from sklearn.linear_model import LogisticRegressionCV

In [155]:
logit = LogisticRegressionCV(class_weight=True)
logit.fit(X_train_lg, y_train_lg)
logit_valid_pred_lg = logit.predict_proba(X_valid_lg)[:, 1]

roc_auc_score(y_valid_lg, logit_valid_pred_lg)

0.6933128146018606

In [0]:
w = 0.4
p_xgb = model_xgb.predict_proba(X_test)[:, 1]
p_log = logit.predict_proba(X_test_logit)[:, 1]
xgb_log_pred = w * p_log + (1-w) * p_xgb
pd.Series(xgb_log_pred, name='dep_delayed_15min').to_csv('xgb_log_model_feat.csv', index_label='id', header=True)

In [0]:
!mv xgb_log_model_feat.csv drive/xgb_log_model_2feat.csv

In [97]:
!git clone https://github.com/catboost/catboost.git

Cloning into 'catboost'...
remote: Counting objects: 46633, done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 46633 (delta 127), reused 159 (delta 37), pack-reused 46355
Receiving objects: 100% (46633/46633), 60.41 MiB | 22.72 MiB/s, done.
Resolving deltas: 100% (21732/21732), done.


In [100]:
!pip install catboost

    100% |████████████████████████████████| 33.6MB 35kB/s 
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [78]:
!git clone https://github.com/markov-av/usml.git

Cloning into 'usml'...
remote: Counting objects: 20, done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20 (delta 7), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [0]:
!mv model_2feat.csv usml/model_2feat.csv

In [84]:
!cd usml && git add . && git commit -m 'add submit' && git push


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@2b81491c1989.(none)')


In [85]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive

In [0]:
!google-drive-ocamlfuse drive

In [89]:
!ls

datalab  drive	mlcourse_open  usml


In [117]:
!ls drive

Colab Notebooks  data  model_2feat.csv


In [0]:
!mv model_2feat.csv drive/model_3feat.csv

In [0]:
X_train = pd.DataFrame()
X_train['Month'] = train['Month']
#X_train['route'] = train['route']
X_train['DayofMonth'] = train['DayofMonth']
X_train['DayOfWeek'] = train['DayOfWeek']
X_train['UniqueCarrier'] = train['UniqueCarrier']
X_train['Distance'] = train['Distance']
X_train['DepTime'] = train['DepTime']

In [8]:
X_train.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Distance,DepTime
0,c-8,c-21,c-7,AA,732,1934
1,c-4,c-20,c-3,US,834,1548
2,c-9,c-2,c-5,XE,416,1422
3,c-11,c-25,c-6,OO,872,1015
4,c-10,c-7,c-6,WN,423,1828


In [0]:
X_train_dum = pd.get_dummies(X_train, columns=['Month', 'DayOfWeek', 'DayofMonth', 'UniqueCarrier'])

In [0]:
scaler_lb = StandardScaler()
X_train_scaler = scaler_lb.fit_transform(X_train[['Distance', 'DepTime']])
#X_valid_lb = scaler_lb.transform(X_valid_lb)

In [154]:
type(X_train_scaler), type(X_test_dum)

(numpy.ndarray, pandas.core.frame.DataFrame)

In [0]:
X_train_conc = np.concatenate((pd.DataFrame(X_train_scaler), X_train_dum), axis=1)

In [0]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train_conc, y_train, test_size=0.3, random_state=17)

In [0]:
#X_train_part_lb = pd.get_dummies(X_train_part_lb, columns=['Month', 'DayOfWeek', 'DayofMonth', 'UniqueCarrier', 'route'])
#X_valid_lb = pd.get_dummies(X_valid_lb, columns=['Month', 'DayOfWeek', 'DayofMonth', 'UniqueCarrier', 'route'])

In [16]:
X_train_part.shape, X_valid.shape

((70000, 76), (30000, 76))

In [17]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.6894437038462284

In [0]:
import xgboost

In [0]:
model_xg = xgboost.XGBClassifier(n_estimators=1000, max_depth=4, reg_alpha=1)

In [21]:
model_xg.fit(X_train_part, y_train_part)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [22]:
xgb_valid_pred = model_xg.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, xgb_valid_pred)

0.7304419065228639


*   n_estimators = 500  - 0.7250925201716044
*   n_estimators = 1000  - 0.7279617018336471
*   max_depth = 6, n_esti = 1000 - 0.7280941970810134
*   4/1000/l1l2 regul - 0.7305893700953754




In [0]:
#X_train_part_lb = np.concatenate((X_train_part, X_train_part_lb), axis=1)
#X_valid_lb = np.concatenate((X_valid, X_valid_lb), axis=1)

In [0]:
#test.head()
X_test = pd.DataFrame()
X_test['Month'] = test['Month']
#X_train['route'] = train['route']
X_test['DayofMonth'] = test['DayofMonth']
X_test['DayOfWeek'] = test['DayOfWeek']
X_test['UniqueCarrier'] = test['UniqueCarrier']
X_test['Distance'] = test['Distance']
X_test['DepTime'] = test['DepTime']

In [0]:
X_test_dum = pd.get_dummies(X_test, columns=['Month', 'DayOfWeek', 'DayofMonth', 'UniqueCarrier'])

In [0]:
X_test_scaler = scaler_lb.transform(X_test[['Distance', 'DepTime']])

In [0]:
X_test_conc = np.concatenate((pd.DataFrame(X_test_scaler), X_test_dum), axis=1)

In [38]:
X_test_conc.shape, X_train_conc.shape

((100000, 74), (100000, 76))

In [41]:
X_test_dum.shape, X_train_dum.shape

((100000, 72), (100000, 74))

In [42]:
col = set(X_train_dum.columns) - set(X_test_dum.columns)
col

{'UniqueCarrier_DH', 'UniqueCarrier_HP', 'UniqueCarrier_TZ'}

In [43]:
set([1, 2, 3]) - set([3, 4, 2])

{1}

In [40]:
X_test.shape, X_train.shape

((100000, 6), (100000, 6))

In [29]:
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test_dum)

model_test_pred = model_xg.predict_proba(X_test_conc)[:, 1]

pd.Series(model_test_pred, name='dep_delayed_15min').to_csv('model_2feat.csv', index_label='id', header=True)

ValueError: ignored

In [0]:
train_lb = pd.DataFrame()
train_lb['Month'] = train['Month'].values
train_lb['route'] = train['route'].values
train_lb['DayofMonth'] = train['DayofMonth'].values
train_lb['DayOfWeek'] = train['DayOfWeek'].values
train_lb['UniqueCarrier'] = train['UniqueCarrier'].values
#train_lb['Distance'] = train['Distance'].values
#train_lb['DepTime'] = train['DepTime'].values

In [12]:
train_lb.head()

,Month,route,DayofMonth,DayOfWeek,UniqueCarrier
0,c-8,ATL-DFW,c-21,c-7,AA
1,c-4,PIT-MCO,c-20,c-3,US
2,c-9,RDU-CLE,c-2,c-5,XE
3,c-11,DEN-MEM,c-25,c-6,OO
4,c-10,MDW-OMA,c-7,c-6,WN


In [0]:
from sklearn import preprocessing

In [0]:
lb = preprocessing.LabelBinarizer()

In [0]:
model_lb_tr_Month = lb.fit_transform(train_lb.Month)
model_lb_tr_route = lb.fit_transform(train_lb.route)
model_lb_tr_DayMonth = lb.fit_transform(train_lb.DayofMonth)
model_lb_tr_dayWeek = lb.fit_transform(train_lb.DayOfWeek)
model_lb_tr_UC = lb.fit_transform(train_lb.UniqueCarrier)

In [0]:
train_lb_concat = np.concatenate((model_lb_tr_Month, 
                                  model_lb_tr_route,
                                  model_lb_tr_DayMonth,
                                  model_lb_tr_dayWeek,
                                  model_lb_tr_UC), axis=1)

In [26]:
train_lb_concat.shape

(100000, 4501)

In [0]:
X_train_part_lb, X_valid_lb, y_train_part_lb, y_valid_lb = train_test_split(train_lb_concat, y_train, test_size=0.3, random_state=17)

In [0]:
logit_lb = LogisticRegression()

In [0]:
logit_lb.fit(X_train_part_lb, y_train_part_lb)